In [2]:
import requests
import pandas as pd
import numpy as np
import glob
import os
import re
import time
import pandas as pd
from apify_client import ApifyClient
from bs4 import BeautifulSoup
import re
import json
from requests.exceptions import HTTPError, ConnectionError
from urllib3.exceptions import ProtocolError


### Extracting Raw Data & Cleaning 

In [5]:
# Load the raw data from Excel file
df = pd.read_excel('Raw_Data/Adhoc_Leads_1.xlsx')

# Reformat: Convert all column names to uppercase
df.columns = df.columns.str.upper()

# Reformat: Strip whitespace from column names
df.columns = df.columns.str.strip()

# Reformat: Replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_', regex=False)

# Reformat: Replace hyphens with underscores
df.columns = df.columns.str.replace('-', '_', regex=False)

# Display basic information about the dataframe
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()


Shape: (1674, 11)

Columns: ['COMPANY_NAME', 'ADDRESS', 'CUSTOMER_NAME', 'EMAIL_ADDRESS', 'CONTACT_NUMBER', 'PRIMARY_SSIC', 'SECONDARY_SSIC', 'SUB_INDUSTRY', 'PIC_NAMES', 'EXTRACTED_PHONE_NUM', 'PIC_EMAIL__ADDRESS']

First few rows:


,COMPANY_NAME,ADDRESS,CUSTOMER_NAME,EMAIL_ADDRESS,CONTACT_NUMBER,PRIMARY_SSIC,SECONDARY_SSIC,SUB_INDUSTRY,PIC_NAMES,EXTRACTED_PHONE_NUM,PIC_EMAIL__ADDRESS
0,CHAI YU TRADING,445 SIN MING AVENUE #01-473 SNG 570445,Yap Chai Ting \n 98381812,NaN,98381812,47802.0,Na,"Wet Markets, Market Stalls & Street Retailers",YAP CHAI TING,98381812,NaN
1,38 VEGETARIAN,"CAUSEWAY POINT, 1, WOODLANDS SQUARE, #B1-K17, ...",Ms. Jenny Ho 9839 5558,NaN,NaN,47219.0,56140,"Wet Markets, Market Stalls & Street Retailers",MS. JENNY HO,98395558,NaN
2,FIRST STATION (T1) PTE. LTD.,HOUSE 80 AIRPORT BOULEVARD MAIN CHANGI AIRPORT...,MS SHIRLEY 98265862 64456030 choicefood@ymail.com,choicefood@ymail.com,98265862,47219.0,na,"Wet Markets, Market Stalls & Street Retailers",MS SHIRLEY,98265862 / 64456030,choicefood@ymail.com
3,DE CHENG XIN XING TRADING PTE LTD,1775 GEYLANG BAHRU #01-03 Singapore 339704,MS HELEN CHEUNG LIU SUK MS NICOLE 63772683 972...,heng328@singnet.com.sg,63772683,47219.0,56111,"Wet Markets, Market Stalls & Street Retailers",MS HELEN CHEUNG LIU SUK MS NICOLE,63772683 / 97262668,heng328@singnet.com.sg
4,COCOA COLONY (S) PTE. LTD.,HOUSE 1 VISTA EXCHANGE GREEN #01-50 THE STAR S...,Mr. Kenny Koh Ms. Wendy 9106 1958 9693 120,NaN,NaN,47213.0,56112,"Wet Markets, Market Stalls & Street Retailers",MR. KENNY KOH MS. WENDY,91061958,NaN


### Getting ACRA DATA

In [8]:

folder_path = "Acra_Data"

# Get all CSV file paths inside the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Read and combine all CSVs
# Using low_memory=False to avoid DtypeWarning for mixed types
df = pd.concat((pd.read_csv(f, low_memory=False) for f in csv_files), ignore_index=True)


df.columns = df.columns.str.upper()


acra_data = df[[
    "UEN",
    "ENTITY_NAME",
    "BUSINESS_CONSTITUTION_DESCRIPTION",
    "ENTITY_TYPE_DESCRIPTION",
    "ENTITY_STATUS_DESCRIPTION",
    "REGISTRATION_INCORPORATION_DATE",
    "PRIMARY_SSIC_CODE",
    "SECONDARY_SSIC_CODE",
    "STREET_NAME",
    "POSTAL_CODE"
]].copy()

# Convert to proper data types
acra_data['UEN'] = acra_data['UEN'].astype('string')
acra_data['ENTITY_NAME'] = acra_data['ENTITY_NAME'].astype('string')
acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'] = acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'].astype('string')
acra_data['ENTITY_TYPE_DESCRIPTION'] = acra_data['ENTITY_TYPE_DESCRIPTION'].astype('string')
acra_data['ENTITY_STATUS_DESCRIPTION'] = acra_data['ENTITY_STATUS_DESCRIPTION'].astype('string')
acra_data['REGISTRATION_INCORPORATION_DATE'] = pd.to_datetime(acra_data['REGISTRATION_INCORPORATION_DATE'], errors='coerce')

# Clean string columns — trim, remove extra spaces, uppercase
for col in [
    'UEN',
    'ENTITY_NAME',
    'BUSINESS_CONSTITUTION_DESCRIPTION',
    'ENTITY_TYPE_DESCRIPTION',
    'ENTITY_STATUS_DESCRIPTION',
    'STREET_NAME',
    'POSTAL_CODE'
]:
    acra_data[col] = (
        acra_data[col]
        .fillna('')
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)
        .str.upper()
    )

# Replace placeholders with NaN for standardization
acra_data.replace(['NA', 'N/A', '-', ''], np.nan, inplace=True)

# Convert registration date to dd-mm-yyyy string (optional)
acra_data['REGISTRATION_INCORPORATION_DATE'] = acra_data['REGISTRATION_INCORPORATION_DATE'].dt.strftime('%d-%m-%Y')

# Filter only live entities (LIVE COMPANY or LIVE)
acra_data = acra_data[
    acra_data['ENTITY_STATUS_DESCRIPTION'].isin(['LIVE COMPANY', 'LIVE'])
].reset_index(drop=True)

# Exclude specific PRIMARY_SSIC_CODE values (supposedly the data would be 600k plus but when we exclude this would lessen)
exclude_codes = [
    46900, 47719, 47749, 47539, 47536, 56123,
    10711, 10712, 10719, 10732, 10733, 93209
]

acra_data = acra_data[~acra_data['PRIMARY_SSIC_CODE'].isin(exclude_codes)].reset_index(drop=True)

In [9]:
acra_data

,UEN,ENTITY_NAME,BUSINESS_CONSTITUTION_DESCRIPTION,ENTITY_TYPE_DESCRIPTION,ENTITY_STATUS_DESCRIPTION,REGISTRATION_INCORPORATION_DATE,PRIMARY_SSIC_CODE,SECONDARY_SSIC_CODE,STREET_NAME,POSTAL_CODE
0,00182000A,AIK SENG HENG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,07-02-1975,46302,na,FISHERY PORT ROAD,619742
1,00233500W,ASIA STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,28-10-1974,46411,20234,SIMS AVENUE,387509
2,00733000J,AIK CHE HIONG,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,02-11-1974,32909,46900,ANG MO KIO INDUSTRIAL PARK 2A,568049
3,00927000X,A WALIMOHAMED BROS,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,12-11-1974,46411,66126,JELLICOE ROAD,208767
4,01173000E,ANG TECK MOH DEPARTMENT STORE,PARTNERSHIP,SOLE PROPRIETORSHIP/ PARTNERSHIP,LIVE,30-10-1974,47711,47214,WOODLANDS STREET 12,738623
...,...,...,...,...,...,...,...,...,...,...
537323,T25LL0518K,ZEUS BARBERS LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,16-05-2025,96021,na,KELANTAN LANE,200031
537324,T25LL0858C,ZENSE SPACE LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,01-08-2025,43301,46900,YISHUN INDUSTRIAL STREET 1,768161
537325,T25LL0870A,ZIQZEQ PROCUREMENT LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,04-08-2025,70209,46100,SIN MING LANE,573969
537326,T25LL1049B,ZHONG XIN TRAVEL LLP,<NA>,LIMITED LIABILITY PARTNERSHIP,LIVE,08-09-2025,79102,79101,JALAN BAHAGIA,320034


In [8]:
# Load the raw data from Excel file
df = pd.read_excel('Raw_Data/Adhoc_Leads_1.xlsx')

# Reformat: Convert all column names to uppercase
df.columns = df.columns.str.upper()

# Reformat: Strip whitespace from column names
df.columns = df.columns.str.strip()

# Reformat: Replace spaces with underscores
df.columns = df.columns.str.replace(' ', '_', regex=False)

# Reformat: Replace hyphens with underscores
df.columns = df.columns.str.replace('-', '_', regex=False)

# Keep only the required columns
df = df[['COMPANY_NAME', 'ADDRESS', 'CUSTOMER_NAME']]

# Extract email addresses from CUSTOMER_NAME column
# Email pattern: word characters, dots, hyphens, plus signs @ domain
email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
df['EXTRACTED_EMAIL'] = df['CUSTOMER_NAME'].str.extract(f'({email_pattern})', expand=False)

# Extract phone numbers from CUSTOMER_NAME column
# Phone pattern: handles various formats like +65 1234 5678, 1234-5678, (65) 12345678, etc.
phone_pattern = r'(?:\+?\d{1,3}[\s.-]?)?\(?\d{3,4}\)?[\s.-]?\d{3,4}[\s.-]?\d{3,4}'
df['EXTRACTED_PHONE'] = df['CUSTOMER_NAME'].str.extract(f'({phone_pattern})', expand=False)

# Display the columns and preview the data
print(f"Columns: {df.columns.tolist()}")
print(f"\nShape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

Columns: ['COMPANY_NAME', 'ADDRESS', 'CUSTOMER_NAME', 'EXTRACTED_EMAIL', 'EXTRACTED_PHONE']

Shape: (1674, 5)

First few rows:


,COMPANY_NAME,ADDRESS,CUSTOMER_NAME,EXTRACTED_EMAIL,EXTRACTED_PHONE
0,CHAI YU TRADING,445 SIN MING AVENUE #01-473 SNG 570445,Yap Chai Ting \n 98381812,NaN,NaN
1,38 VEGETARIAN,"CAUSEWAY POINT, 1, WOODLANDS SQUARE, #B1-K17, ...",Ms. Jenny Ho 9839 5558,NaN,NaN
2,FIRST STATION (T1) PTE. LTD.,HOUSE 80 AIRPORT BOULEVARD MAIN CHANGI AIRPORT...,MS SHIRLEY 98265862 64456030 choicefood@ymail.com,choicefood@ymail.com,98265862 6445
3,DE CHENG XIN XING TRADING PTE LTD,1775 GEYLANG BAHRU #01-03 Singapore 339704,MS HELEN CHEUNG LIU SUK MS NICOLE 63772683 972...,heng328@singnet.com.sg,63772683 9726
4,COCOA COLONY (S) PTE. LTD.,HOUSE 1 VISTA EXCHANGE GREEN #01-50 THE STAR S...,Mr. Kenny Koh Ms. Wendy 9106 1958 9693 120,NaN,9106 1958 9693
